In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import e

In [52]:
def forward(W1, W2, b1, b2, x):
    capa1 = W1 @ x + b1
    
    m = capa1.shape[0]
    
    for i in range(m):
        capa1[i][0] = max(0, capa1[i][0])

    capa2 = W2 @ capa1 + b2
    return capa2[0][0]


In [53]:
#Inicializacion de pesos

W1 = np.random.random((5, 6))  # matriz 5x6 
b1 = np.random.random((5, 1))

W2 = np.random.random((1, 5))
b2 = np.random.random((1, 1))

In [54]:
#Calculo del gradiente numerico
def funcion_objetivo(x, y, W1, W2, b1, b2):
    loss = 0.5*((forward(W1,W2,b1,b2,x.transpose())-y)**2)
    return loss

def numerical_gradient(x, y, W1, W2, b1, b2, eps):
    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            W1_izq = W1 
            W1_izq[i][j] = W1_izq[i][j] + eps
            W1_der = W1 
            W1_der[i][j] = W1_der[i][j] - eps
            izq_W1 = funcion_objetivo(x, y, W1_izq, W2, b1, b2)
            der_W1 = funcion_objetivo(x, y, W1_der, W2, b1, b2)
            derivada_W1 = (izq_W1 - der_W1)/(2*eps)
    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2_izq = W2
            W2_izq[i][j] = W2_izq[i][j] + eps
            W2_der = W2
            W2_der[i][j] = W2_der[i][j] - eps
            izq_W2 = funcion_objetivo(x, y, W1, W2_izq, b1, b2)
            der_W2 = funcion_objetivo(x, y, W1, W2_der, b1, b2)
            derivada_W2 = (izq_W2 - der_W2)/(2*eps)
    for i in range(b1.shape[0]):
        for j in range(b1.shape[1]):
            b1_izq = b1
            b1_izq[i][j] = b1_izq[i][j] + eps
            b1_der = b1
            b1_der[i][j] = b1_der[i][j] - eps
            izq_b1 = funcion_objetivo(x, y, W1, W2, b1_izq, b2)
            der_b1 = funcion_objetivo(x, y, W1, W2, b1_der, b2)
            derivada_b1 = (izq_b1 - der_b1)/(2*eps)
    for i in range(b2.shape[0]):
        for j in range(b2.shape[1]):
            b2_izq = b2
            b2_izq[i][j] = b2_izq[i][j] + eps
            b2_der = b2
            b2_der[i][j] = b2_der[i][j] - eps
            izq_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_der)
            der_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_izq)
            derivada_b2 = (izq_b2 - der_b2)/(2*eps)
    
    gradiente = np.zeros((4,1))
    gradiente[0][0] = derivada_W1
    gradiente[1][0] = derivada_W2
    gradiente[2][0] = derivada_b1
    gradiente[3][0] = derivada_b2
    
    return gradiente


In [55]:
# #Calculo del gradiente numerico
# def funcion_objetivo(x, y, W1, W2, b1, b2):
#     loss = 0.5*((forward(W1,W2,b1,b2,x.transpose())-y)**2)
#     return loss

# def numerical_gradient(x, y, W1, W2, b1, b2, eps):

#     cincremental_der_W1 = funcion_objetivo(x, y, W1+eps, W2, b1, b2)
#     cincremental_izq_W1 = funcion_objetivo(x, y, W1-eps, W2, b1, b2)
#     derivada_W1 = (cincremental_der_W1-cincremental_izq_W1)/(2*eps)

#     cincremental_der_W2 = funcion_objetivo(x, y, W1, W2+eps, b1, b2)
#     cincremental_izq_W2 = funcion_objetivo(x, y, W1, W2-eps, b1, b2)
#     derivada_W2 = (cincremental_der_W2-cincremental_izq_W2)/(2*eps)

#     cicremental_der_b1 = funcion_objetivo(x, y, W1, W2, b1 + eps, b2)
#     cicremental_izq_b1 = funcion_objetivo(x, y, W1, W2, b1 - eps, b2)
#     derivada_b1 = (cicremental_der_b1 - cicremental_izq_b1)/(2*eps)

#     cicremental_der_b2 = funcion_objetivo(x, y, W1, W2, b1, b2 + eps)
#     cicremental_izq_b2 = funcion_objetivo(x, y, W1, W2, b1, b2 - eps)
#     derivada_b2 = (cicremental_der_b2 - cicremental_izq_b2)/(2*eps)

#     gradiente = np.zeros((4,1))
#     gradiente[0][0] = derivada_W1
#     gradiente[1][0] = derivada_W2
#     gradiente[2][0] = derivada_b1
#     gradiente[3][0] = derivada_b2
    
#     return gradiente

In [56]:
#funcion fit y loop de entrenamiento
def fit(x, y, W1, W2, b1, b2, learning_rate, epochs):
    eps = 1e-3
    loss_accum = []
    j = 0
    gradw1 = 0
    gradw2 = 0
    gradb1 = 0
    gradb2 = 0
    for i in range(epochs):
        aux_loss = 0
        for j in range(x.shape[0]):
            x_aux = np.array([x[j,:]])
            objective_value = funcion_objetivo(x_aux, y[j], W1, W2, b1, b2)
            aux_loss = aux_loss + objective_value

            gradiente = numerical_gradient(x_aux, y[j], W1, W2, b1, b2, eps)
            gradw1 = gradw1 + gradiente[0]
            gradw2 = gradw2 + gradiente[1]
            gradb1 = gradb1 + gradiente[2]
            gradb2 = gradb2 + gradiente[3]

        gradw1 = gradw1/x.shape[0]
        gradw2 = gradw2/x.shape[0]
        gradb1 = gradb1/x.shape[0]
        gradb2 = gradb2/x.shape[0]

        W1 = W1 - learning_rate*gradw1
        W2 = W2 - learning_rate*gradw2
        b1 = b1 - learning_rate*gradb1
        b2 = b2 - learning_rate*gradb2

        loss_accum.append(aux_loss/x.shape[0])

    theta = [W1, W2, b1, b2]
    return theta, loss_accum

In [57]:
#funcion fit y loop de entrenamiento

def fit_2(x, y, W1, W2, b1, b2, learning_rate, epochs):
    eps = 1e-3
    loss_accum = []
    j = 0
    grad_w1 = np.zeros(W1.shape)
    grad_w2 = np.zeros(W2.shape)
    grad_b1 = np.zeros(b1.shape)
    grad_b2 = np.zeros(b2.shape)
    for i in range(epochs):
        aux_loss = 0
        for j in range(x.shape[0]):
            x_aux = np.array([x[j,:]])

            objective_value = funcion_objetivo(x_aux, y[j], W1, W2, b1, b2)

            aux_loss = aux_loss + objective_value

            gradiente = numerical_gradient(x_aux, y[j], W1, W2, b1, b2, eps)
            grad_w1 = grad_w1 + gradiente[0]
            grad_w2 = grad_w2 + gradiente[1]
            grad_b1 = grad_b1 + gradiente[2]
            grad_b2 = grad_b2 + gradiente[3]

        loss_accum.append(aux_loss/x.shape[0])

        grad_w1 = grad_w1 / x.shape[0]
        grad_w2 = grad_w2 / x.shape[0]
        grad_b1 = grad_b1 / x.shape[0]
        grad_b2 = grad_b2 / x.shape[0]

        W1 = W1 - learning_rate * grad_w1
        W2 = W2 - learning_rate * grad_w2
        b1 = b1 - learning_rate * grad_b1
        b2 = b2 - learning_rate * grad_b2

    theta = [W1, W2, b1, b2]
    return theta, loss_accum

In [58]:
def predict(x, W1, W2, b1, b2):
    y = forward(W1, W2, b1, b2, x)
    return y


In [59]:
def normalizar_x(x):
    medias = x.mean()
    for i in range(x.shape[0]):
        x[i] = x[i] - medias
    return x  

def normalizar_y(y):
    media = y.mean()
    for i in range(y.shape[0]):
        y[i] = y[i] - media
    return y

In [61]:
df = pd.read_excel("Real estate valuation data set.xlsx")
datos = df.to_numpy()

X = datos[:100,[1,2,3,4,5,6]]
Y = datos[:100,7]

X_test = datos[315:, [1,2,3,4,5,6]]
Y_test = datos[315:,7]

# x_norm = normalizar_x(X)
# y_norm = normalizar_y(Y)
theta = fit(X, Y, W1, W2, b1, b2, 0.0005, 1000)[0]
# print(fit(X, Y, W1, W2, b1, b2, 0.0005, 1000)[1])

# x_test_norm = normalizar_x(X_test)
# y_test_norm = normalizar_y(Y_test)

# xt_test_norm = x_test_norm.transpose()

Xt_test = X_test.transpose()

predictions = []
resta = []

# for i in range(X_test.shape[0]): 
#     predictions.append(predict(xt_test_norm[:,i], theta[0], theta[1], theta[2], theta[3]))
#     resta.append(predictions[i] - y_test_norm[i])
#     print(predictions[i], "- ", y_test_norm[i], "=", resta[i])

for i in range(X_test.shape[0]): 
    predictions.append(predict(Xt_test[:,i], theta[0], theta[1], theta[2], theta[3]))
    resta.append(predictions[i] - Y_test[i])
    print(predictions[i], "- ", Y_test[i], "=", resta[i])


6683.652326260032 -  27.3 = 6656.352326260032
5264.678545964278 -  42.0 = 5222.678545964278
5734.217810743808 -  37.5 = 5696.717810743808
5388.832143114676 -  49.8 = 5339.032143114676
5285.070141063453 -  26.9 = 5258.170141063453
8988.199495362713 -  18.6 = 8969.599495362712
5936.6827497154945 -  37.7 = 5898.982749715495
5199.545911388926 -  33.1 = 5166.445911388925
5214.58077957634 -  42.5 = 5172.08077957634
6641.056800293488 -  31.3 = 6609.756800293488
5491.521699566861 -  38.1 = 5453.42169956686
5081.228582195139 -  62.1 = 5019.128582195139
5740.282973501894 -  36.7 = 5703.582973501894
6439.3254738322885 -  23.6 = 6415.725473832288
8988.805585573218 -  19.2 = 8969.605585573217
6116.134876182886 -  12.8 = 6103.334876182886
9294.957830123445 -  15.6 = 9279.357830123445
5608.907622227123 -  39.6 = 5569.307622227123
5123.733654529128 -  38.4 = 5085.333654529129
5984.145954569383 -  22.8 = 5961.345954569383
5347.6523464403635 -  36.5 = 5311.1523464403635
6787.217854459165 -  35.6 = 6751.